# Laboratorio 5 - Documenti BBC

Si richiede un'implementazione di un esercizio di Topic Modeling, utilizzando librerie open (come ad es. GenSim (https://radimrehurek.com/gensim/). Si richiede l'utilizzo di un corpus di almeno 1k documenti. Testare un algoritmo (ad esempio LDA) con più valori di k (num. di topics) e valutare la coerenza dei risultati, attraverso fine-tuning su parametri e pre-processing. Update: essendo che spesso i topic, per essere interpretabili, devono contenere content words, potete pensare di filtrare solamente i sostantivi in fase di preprocessing (cioè POS=noun).

In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk import MWETokenizer, WordNetLemmatizer
from nltk.corpus import wordnet as wn, stopwords
import gensim
from gensim import corpora

## Preprocessing delle frasi

In [2]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

def preprocessing(text):
    text = re.sub(r'[^\w\s]',' ',text) # rimuovo la punteggiatura
    text = text.lower()
    text = nltk.pos_tag(text.split()) # prendo i pos tag delle parole (fa anche il tokenizing)
    text = [x for x in text if x[1] in ['NN','NNS','NNP','NNPS']] # mantengo solo i noun
    text = [x[0] for x in text] # rimuovo i pos tag
    text = [lemmatizer.lemmatize(x) for x in text]
    text = [x for x in text if x not in stop_words]
    return text

## Prelevo i documenti

In [3]:
paths = [
    "documents\\bbc\\entertainment", 
    "documents\\bbcsport\\athletics", 
    "documents\\bbcsport\\cricket", 
    "documents\\bbcsport\\football", 
    "documents\\bbcsport\\rugby",
    "documents\\bbcsport\\tennis"
]

documents = []

for path in paths:
    print(path + ' contiene ' + str(len(os.listdir(path))) + ' files')
    for file_name in os.listdir(path):
        if os.path.isfile(os.path.join(path, file_name)):
            file = open(path + "/" + file_name, "r", encoding="utf-8")
            document = preprocessing(file.read())
            documents.append(document)

documents\bbc\entertainment contiene 128 files
documents\bbcsport\athletics contiene 101 files
documents\bbcsport\cricket contiene 124 files
documents\bbcsport\football contiene 265 files
documents\bbcsport\rugby contiene 147 files
documents\bbcsport\tennis contiene 100 files


## LDA

In [4]:
"""
I valori dell'output rappresentano i punteggi di coerenza associati ai rispettivi modelli LDA. 
Questi punteggi sono stati ottenuti mediante l'utilizzo del CoherenceModel. Esso calcola la coerenza 
dei topic utilizzando diverse metriche di coerenza, come ad esempio la metrica C_V. 
Queste metriche valutano la somiglianza semantica tra le parole chiave all'interno di ciascun topic, 
considerando anche la distribuzione delle parole nel corpus di testo.
"""
def fine_tuning(texts, limit, start=2, step=1):
    coherence_values = []
    model_list = []
    num_topic_list = []

    dictionary = corpora.Dictionary(texts)
    dictionary.filter_extremes(no_below=5, no_above=0.3, keep_n=None)  # use Dictionary to remove un-relevant tokens
    corpus = [dictionary.doc2bow(doc) for doc in texts]


    for num_topics in range(start, limit, step):
        print("Elaborazione per il topic " + str(num_topics))
        model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        num_topic_list.append(num_topics)
        coherencemodel = gensim.models.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, num_topic_list, coherence_values

In [5]:
model_list, num_topic_list, coherence_values = fine_tuning(documents, 10)

Elaborazione per il topic 2
Elaborazione per il topic 3
Elaborazione per il topic 4
Elaborazione per il topic 5
Elaborazione per il topic 6
Elaborazione per il topic 7
Elaborazione per il topic 8
Elaborazione per il topic 9


In [8]:
for i in range(0, len(num_topic_list)):
    print(str(num_topic_list[i]) + " topic : " + str(coherence_values[i]))

2 topic : 0.3385441027087843
3 topic : 0.35718001168050173
4 topic : 0.39613896904985524
5 topic : 0.33789973447331806
6 topic : 0.34943276719709826
7 topic : 0.3277788731532742
8 topic : 0.3232278732396564
9 topic : 0.3320317079282563


In [9]:
for i in range(0, len(num_topic_list)):
    print(f"Modello con {num_topic_list[i]} topic: \n")
    for topic_id, topic in model_list[i].show_topics(formatted=True, num_topics=num_topic_list[i], num_words=10):
        print(f"Topic {topic_id}: {topic}")
    print('\n')

Modello con 2 topic: 

Topic 0: 0.010*"england" + 0.009*"club" + 0.008*"cup" + 0.007*"test" + 0.006*"season" + 0.006*"injury" + 0.006*"chelsea" + 0.005*"ball" + 0.005*"minute" + 0.005*"series"
Topic 1: 0.007*"film" + 0.006*"england" + 0.006*"wale" + 0.005*"champion" + 0.005*"goal" + 0.005*"coach" + 0.005*"ball" + 0.005*"test" + 0.005*"week" + 0.005*"point"


Modello con 3 topic: 

Topic 0: 0.009*"club" + 0.008*"england" + 0.007*"chelsea" + 0.007*"champion" + 0.007*"cup" + 0.007*"season" + 0.006*"test" + 0.006*"injury" + 0.005*"number" + 0.005*"series"
Topic 1: 0.010*"film" + 0.008*"goal" + 0.007*"club" + 0.007*"minute" + 0.007*"cup" + 0.006*"award" + 0.006*"england" + 0.005*"chance" + 0.005*"season" + 0.005*"half"
Topic 2: 0.011*"england" + 0.010*"ball" + 0.008*"test" + 0.006*"coach" + 0.006*"week" + 0.006*"wale" + 0.006*"series" + 0.006*"half" + 0.005*"ireland" + 0.005*"nation"


Modello con 4 topic: 

Topic 0: 0.009*"ball" + 0.007*"test" + 0.006*"season" + 0.006*"champion" + 0.006*"w